# Graded Challange 3

Hi, my name Fauzan Nurachman a.k.a Ojan from class Full Time Data Science Batch 003

## Description

### Objective

You are a data analyst at The Look which is one of the largest e-commerce platforms on the planet Mars. You are asked to create an evaluation report on sales activities on the platform.

## Working Area

Importing Library

We will import libraries from google.colab, google.cloud and pandas to retrieve data and process the data we want.

In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd

auth.authenticate_user()
print('Authenticated')

project_id = "hacktiv8-fauzan" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


### Assignment Problem

Poin penjabaran:

1. Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?
2. Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete. Apa informasi yang bisa kamu sampaikan?
3. Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?
4. Kategori produk apa saja yang paling banyak dibeli (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!
5. Kategori produk apa saja yang paling banyak mendapatkan pendapatan (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!
6. Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan? (Hint: Kamu bisa menggunakan korelasi)

POIN ANALISIS

Berikan kesimpulan dari laporan/informasi yang dibuat berdasarkan problem statement/poin penjabaran menggunakan bahasa awam! (boleh dalam beberapa paragraf)

POIN PERTANYAAN

1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!
2. Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?
3. CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!

### Answer

Poin Penjabaran:

1. We will create a query in Google Cloud Platform with a query like the following:

In [ ]:
complete_transaction = client.query('''
SELECT Year, Month, Complete,
  CASE WHEN month BETWEEN 1 AND 3 THEN 1
       WHEN month BETWEEN 4 AND 6 THEN 2
       WHEN month BETWEEN 7 AND 9 THEN 3
  END AS Quarter
FROM ( SELECT 
       EXTRACT(year from created_at) as Year, 
       EXTRACT(month from created_at) as Month, 
       COUNT(status) as Complete
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE created_at >= '2022-01-01' AND created_at <= '2022-09-30' and status = 'Complete' 
GROUP BY year,month
ORDER BY year,month)
''').to_dataframe()

In [ ]:
complete_transaction

,Year,Month,Complete,Quarter
0,2022,1,1316,1
1,2022,2,1364,1
2,2022,3,1564,1
3,2022,4,1635,2
4,2022,5,1750,2
5,2022,6,1879,2
6,2022,7,2111,3
7,2022,8,2378,3
8,2022,9,2355,3


In the table above we can find out the number of complete transactions per month during Q1 to Q3. Before we make this query, we take data from order.items, then bring the logic by taking transaction data that has been completed, then we extract years and months to make it easier for us to determine the number of complete and quartal, then we have the number of complete per month, after getting the number of transactions per month we make perquartal by dividing per 3 months into 1 quartal.

We can see the table above that the trending number of complete transactions per month has increased from Q1 to Q3 (although there was a slight decrease in February and June) but at the end of Q3 in September to 2520 completed transactions.

2. We will create a query in Google Cloud Platform with a query like the following:

In [ ]:
total_sale = client.query('''
SELECT Year, Month,
  CASE WHEN month BETWEEN 1 AND 3 THEN 1
       WHEN month BETWEEN 4 AND 6 THEN 2
       WHEN month BETWEEN 7 AND 9 THEN 3
  END AS Quarter,
  round(Total_Sales) AS Total_Sales
FROM ( SELECT 
       EXTRACT(year from items.created_at) as Year, 
       EXTRACT(month from items.created_at) as Month, 
       SUM(items.sale_price) as Total_Sales
FROM `bigquery-public-data.thelook_ecommerce.order_items` AS items
JOIN `bigquery-public-data.thelook_ecommerce.inventory_items` AS inventory ON items.product_id = inventory.product_id
WHERE items.created_at >= '2022-01-01' AND items.created_at <= '2022-09-30' AND status = 'Complete'
GROUP BY year,month
ORDER BY year,month)
''').to_dataframe()

In [ ]:
total_sale

,Year,Month,Quarter,Total_Sales
0,2022,1,1,1597907.0
1,2022,2,1,1698619.0
2,2022,3,1,1843360.0
3,2022,4,2,2064070.0
4,2022,5,2,1977405.0
5,2022,6,2,2199728.0
6,2022,7,3,2456280.0
7,2022,8,3,2701233.0
8,2022,9,3,2761943.0


In the table above we can find out the total sales that are complete each month during Q1 to Q3. Before we create this query, we take data from order.items and inventory.items. Then we can join inventory.items and take product_id as a foreign key, then bring the logic by taking the total sales that have been completed, then we extract the years and months to make it easier for us to determine the total sales of complete and quartal, then we total sales complete per month, after getting the total sales per month we make perquartal by dividing per 3 months into 1 quartal.

We can see the table above that the trending total sales completed per month increased from Q1 to Q3 (although there was a slight decrease in February) but at the end of Q3 in September the total sales to 2,851,957 USD.

3. We will create a query in Google Cloud Platform with a query like the following:

In [ ]:
total_users = client.query('''
SELECT Year, Month,
  CASE WHEN month BETWEEN 1 AND 3 THEN 1
       WHEN month BETWEEN 4 AND 6 THEN 2
       WHEN month BETWEEN 7 AND 9 THEN 3
  END AS Quarter, 
  Total_Users
FROM (  SELECT 
        EXTRACT(year from created_at) as Year, 
        EXTRACT(month from created_at) as Month, 
        COUNT(DISTINCT user_id) as Total_Users
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE created_at >= '2022-01-01' AND created_at <= '2022-09-30' AND status = 'Complete'
GROUP BY year,month
ORDER BY year,month)
''').to_dataframe()

In [ ]:
total_users

,Year,Month,Quarter,Total_Users
0,2022,1,1,909
1,2022,2,1,941
2,2022,3,1,1074
3,2022,4,2,1150
4,2022,5,2,1252
5,2022,6,2,1338
6,2022,7,3,1462
7,2022,8,3,1640
8,2022,9,3,1655


In the table above we can find out how many users complete each month during Q1 to Q3. Before that, we take this query from order.items, then bring the logic by taking complete user_id data, then we extract the year and month to make it easier for us to determine how many users and quartals, then we calculate how many users are complete per month, after getting how many users per month we make perquartal by dividing per 3 months into 1 quartal.

We can see the table above that the trending number of complete users per month has increased from Q1 to Q3 (although there was a slight decrease in February) but at the end of Q3 in September to 1702 user complete.

4. We will create a query in Google Cloud Platform with a query like the following:

In [ ]:
Category_Price = client.query('''
SELECT Year, Month, Quarter, Category
FROM (SELECT *, ROW_NUMBER() OVER(PARTITION BY month ORDER BY Total_Sale_Price DESC) AS row,
CASE WHEN month BETWEEN 1 AND 3 THEN 1
     WHEN month BETWEEN 4 AND 6 THEN 2
     WHEN month BETWEEN 7 AND 9 THEN 3
     END AS Quarter,
FROM (SELECT 
EXTRACT(year from items.created_at) as Year, 
EXTRACT(month from items.created_at) as Month, 
SUM(items.sale_price) as Total_Sale_Price, 
inventory.product_category as Category
FROM `bigquery-public-data.thelook_ecommerce.order_items` AS items
JOIN `bigquery-public-data.thelook_ecommerce.inventory_items` AS inventory ON items.product_id = inventory.product_id
WHERE items.created_at >= '2022-01-01' AND items.created_at <= '2022-09-30' AND items.status = 'Complete' 
GROUP BY category, year, month))
WHERE row = 1
ORDER BY month, year
''').to_dataframe()

In [ ]:
Category_Price

,Year,Month,Quarter,Category
0,2022,1,1,Outerwear & Coats
1,2022,2,1,Jeans
2,2022,3,1,Outerwear & Coats
3,2022,4,2,Outerwear & Coats
4,2022,5,2,Jeans
5,2022,6,2,Jeans
6,2022,7,3,Outerwear & Coats
7,2022,8,3,Jeans
8,2022,9,3,Outerwear & Coats


In the table above we can find out what categories of products are purchased with complete conditions every month during Q1 to Q3. Before we make this query, we take data from order.items and inventory.items, then we take the foreign key, which is product.id to get the data we want, then bring the logic by taking Total_Sale_Price as our reference that how many people buy the product, then we look for user_id data that has been completed, then we extract the year and month to make it easier for us to determine how to get product categories per month and quartal,  Then we calculate the complete Total_Sale_Price per month to get category products, after getting many buyers based on product categories per month we make perquartal by dividing per 3 months into 1 quartal.

We can see the table above that the most purchased product category is Jeans for 7 months, in February and May buyers buy a lot of Outwear & Coats products. We can conclude in Q1 - Q3 buyers are more dominant buying Jeans.

5. We will create a query in Google Cloud Platform with a query like the following:

In [ ]:
Category_Income = client.query('''
SELECT Year, Month, Quarter, Categories
FROM
(SELECT *, ROW_NUMBER() OVER(PARTITION BY month ORDER BY Total_Income_Transaction DESC) AS row,
CASE WHEN month BETWEEN 1 AND 3 THEN 1
     WHEN month BETWEEN 4 AND 6 THEN 2
     WHEN month BETWEEN 7 AND 9 THEN 3
END AS Quarter
FROM (SELECT 
EXTRACT(year from items.created_at) AS Year,
EXTRACT(month from items.created_at) AS Month,  
COUNT(items.created_at) AS Total_Income_Transaction,
inventory.product_category AS Categories, 
FROM `bigquery-public-data.thelook_ecommerce.order_items` AS items
JOIN `bigquery-public-data.thelook_ecommerce.inventory_items` AS inventory ON items.product_id = inventory.product_id
WHERE items.created_at >= '2022-01-01' AND items.created_at <= '2022-09-30' AND items.status = 'Complete'
GROUP BY Categories, Month, Year))
WHERE row = 1
ORDER BY Month, Year
''').to_dataframe()

In [ ]:
Category_Income

,Year,Month,Quarter,Categories
0,2022,1,1,Jeans
1,2022,2,1,Swim
2,2022,3,1,Tops & Tees
3,2022,4,2,Tops & Tees
4,2022,5,2,Intimates
5,2022,6,2,Jeans
6,2022,7,3,Jeans
7,2022,8,3,Shorts
8,2022,9,3,Jeans


In the table above we can find out what product category is based on revenue with complete conditions every month during Q1 to Q3. Before we make this query, we take data from order.items and inventory.items, then we take the foreign key, which is product.id to get the data we want, then bring the logic by taking Total_Income_Transaction as our reference that how much income from the product category, then we look for user_id data that has been completed, then we extract the year and month to make it easier for us to later determine getting the product category per month and  quartal, then we calculate the complete Total_Income_Transaction per month to get category products, after getting income based on the monthly product category we make perquartal by dividing per 3 months into 1 quartal.


We can see the table above that the product categories based on monthly income vary. However, Jeans and Intimates became the biggest contributors for 3 months for the product category, then for others such as Sweaters, Tops & Tees, and Fashion Hoodies & Sweatshirts contributed 1 month each to the product category based on revenue.

6. Before we analyze using correlation, we start by calling the python library that is pandas to get a visualization:

In [ ]:
import pandas as pd

Then we combine between 2 columns, namely the completed transaction column and the total users, then we also enter the total sales so that it occurs with the correlation we want.

In [ ]:
total_sale['complete_transaction'] = complete_transaction['Complete']
total_sale['total_users'] = total_users['Total_Users']

In [ ]:
total_sale

,Year,Month,Quarter,Total_Sales,complete_transaction,total_users
0,2022,1,1,1597907.0,1316,909
1,2022,2,1,1698619.0,1364,941
2,2022,3,1,1843360.0,1564,1074
3,2022,4,2,2064070.0,1635,1150
4,2022,5,2,1977405.0,1750,1252
5,2022,6,2,2199728.0,1879,1338
6,2022,7,3,2456280.0,2111,1462
7,2022,8,3,2701233.0,2378,1640
8,2022,9,3,2761943.0,2355,1655


Then we'll drop columns we don't need like Year, Month, and Quarter. Before that, we first copy the dataframe, then we use the .drop() function to get rid of the columns we don't want.

In [ ]:
correlation = total_sale.copy()
correlation

,Year,Month,Quarter,Total_Sales,complete_transaction,total_users
0,2022,1,1,1597907.0,1316,909
1,2022,2,1,1698619.0,1364,941
2,2022,3,1,1843360.0,1564,1074
3,2022,4,2,2064070.0,1635,1150
4,2022,5,2,1977405.0,1750,1252
5,2022,6,2,2199728.0,1879,1338
6,2022,7,3,2456280.0,2111,1462
7,2022,8,3,2701233.0,2378,1640
8,2022,9,3,2761943.0,2355,1655


In [ ]:
correlation.drop(['Year', 'Month', 'Quarter', ], axis = 1, inplace=True)

Then we do the correlation using the pearson method which means standard correlation coefficient.

In [ ]:
correlation.corr(method = 'pearson')

,Total_Sales,complete_transaction,total_users
Total_Sales,1.000000,0.990021,0.987123
complete_transaction,0.990021,1.000000,0.997679
total_users,0.987123,0.997679,1.000000


Then we see from the table above, from the three columns it can be seen that between the 3 variables, namely total sales, complete transactions, and total users are interdependent on each other. Looking from the correlation value obtained, if the number is close to 1, the more dependent / related to others. We see that everything is almost close to 1 with values between 0.98 - 0.99. So around complete_transaction and total_users its very significant.

#### Problem Statement

In this section, we will conduct an analysis using the SMART method in this company. By using the SMART method we can determine goals, achievements, and organize planning for the future. Here's the SMART model I made: 

* Specific

    We want to see our company trending whether it is good or not.

* Measureable

    We want to see a minimum increase of 50% in sales, total users, and transactions.

* Achieveable

    We want to get the goal of attracting as many users as possible to use our platform and be consistent with our platform.
* Relevant

    We want to maximize revenue during Q1 - Q3 of 2022.

* Time Bound

    For a time of 9 months in 2022 (Q1 - Q3).






Poin Analisis

After we do the analysis from thelook ecommerce, then we create insights from some of the questions needed by the client, we will do the elaboration of the analysis:

1. We have complete buyer interest data in Q1 to Q3: 
   
    Month 1 = 1316

    Month 2 = 1364
   
    Month 3 = 1564
   
    Month 4 = 1635
   
    Month 5 = 1750
   
    Month 6 = 1879
   
    Month 7 = 2111
   
    Month 8 = 2378
   
    Month 9 = 2355

    We can see that starting from month 1 to month 9, there was an increase in buyers of about 80% from the original in month 1 1316 complete buyers to 2355 complete buyers in month 9.

2. We have complete total user data in Q1 to Q3:
  
    month 1 = 909 users

    month 2 = 941 users
   
    month 3 = 1074 users
   
    month 4 = 1150 users
   
    month 5 = 1252 users
   
    month 6 = 1338 users
   
    month 7 = 1462 users
   
    month 8 = 1640 users
   
    month 9 = 1655 users

    We can see that starting from month 1 to month 9, there was an increase in the number of complete users by about 80% from the original in month 1 909 users who were complete to 1655 users who were complete in month 9.

3. We have complete sales data in Q1 to Q3:

    Month 1 = $1,597,907

    Month 2 = $1,698,619
   
    Month 3 = $1,843,360
   
    Month 4 = $2,064,070
   
    Month 5 = $1,977,405
   
    Month 6 = $2,199,728
   
    Month 7 = $2,456,280
   
    Month 8 = $2,701,233
   
    Month 9 = $2,761,943

    We can see that starting from month 1 to month 9, there was an increase in complete sales of around 80% from the original 1st month 1,597,907 USD completed to 2,761,943 USD completed in month 9.

   From the insights above, we can know that there was an increase during Q1 to Q3 of around 80% in the aspects of total completed transactions, total completed users, and total completed sales. Thus we can know that thelook ecommerce is very good in terms of business.

#### POIN PERTANYAAN

1. Yes, in its realization we can exceed the target, where the initial target is at least an increase in the trend of about 50%, but what we get can be up to 80% which is where it can be 1.5x from the initial target.

2. Yes, it makes a lot of sense, because the revenue in our ecommerce in the final Q3 of September was 2,761,943 USD. If the CEO wants to target revenue in Q4 of 250,000 USD it is very easy.

3. We can see from the income we have, that our income has increased according to our target. The percentage increase in revenue, GMV, and users is around 80%, where the initial target is 50%. Therefore, our company is very worthy to get new investors to inject funds into our company.

# TERIMA KASIH